## Scraping descripción de proyectos

In [1]:
!pip install fake_useragent

In [1]:
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import LabelEncoder
from pandas import Series
import math
import csv

In [2]:
import os
os.getcwd()

'C:\\Users\\ALONSO'

In [5]:
os.chdir('D:/TTT/DB/datasets finales')

In [6]:
data_final = pd.read_csv("Data final limpia.csv",sep = ',')
data_final.head()

,id,backers_count,name,blurb,category,photo,urls,city,country,goal,pledged,currency,usd_pledged,created_at,launched_at,deadline,duration,state
0,1000245024,0,Safer Home,Placing furniture against traditional plugs is...,Hardware,https://ksr-ugc.imgix.net/assets/011/663/874/0...,https://www.kickstarter.com/projects/homesafet...,Kamloops,CA,50000.0,0.0,CAD,0.000000,"March 09, 2014","June 03, 2015","July 03, 2015",30,failed
1,1000256230,0,Shreddit - Privacy on Reddit,"Shreddit, a Reddit privacy tool I created and ...",Software,https://ksr-ugc.imgix.net/assets/013/466/903/0...,https://www.kickstarter.com/projects/466914929...,Edinburgh,GB,2000.0,0.0,GBP,0.000000,"August 23, 2016","June 02, 2017","July 02, 2017",30,failed
2,1000261018,3,Diposta - liberating people from their postal ...,The problem of mail: it is physical! Diposta c...,Web,https://ksr-ugc.imgix.net/assets/012/071/808/1...,https://www.kickstarter.com/projects/105350477...,Raleigh,US,100000.0,141.0,USD,141.000000,"March 17, 2015","July 24, 2016","August 23, 2016",30,failed
3,100039820,3,Best Spray Bottle Ever - SureShot,Ever had a spray bottle that has a little bit ...,Gadgets,https://ksr-ugc.imgix.net/assets/012/009/461/3...,https://www.kickstarter.com/projects/110136848...,Edmonton,CA,25000.0,3.0,CAD,2.411101,"January 22, 2015","February 05, 2015","March 07, 2015",30,failed
4,1000468345,6,The iRNinja - Simplify your TV with a wireless...,Control your TV & Audio/Video components by pr...,Gadgets,https://ksr-ugc.imgix.net/assets/016/561/251/1...,https://www.kickstarter.com/projects/5877985/i...,Jupiter,US,500.0,74.0,USD,74.000000,"April 27, 2017","May 10, 2017","June 14, 2017",35,failed


In [7]:
urls_list = data_final["urls"].unique()
#urls_list[63]

In [8]:
from fake_useragent import UserAgent
import urllib.request
from bs4 import BeautifulSoup        #Analizar data
import re

Newlines = re.compile(r'[\r\n]\s+')

In [9]:
def getPageText(url):
    # Crear agente falso para scrapear
    ua = UserAgent()
    user_agent = ua.chrome
    # Solicitar uso de librería urllib con agente falso
    request = urllib.request.Request(url, headers = {"User-Agent":user_agent})
    # Obtener contenido de urls mediante librería urllib
    data = urllib.request.urlopen(request)
    # parse as html structured document
    bs = BeautifulSoup(data, "html.parser")
    # Buscar todos los tags div con una determinada clase
    # A partir de acá, se usa un "try" y un "except" porque hay páginas que no muestran su contenido
    # Para evitar que salga el mensaje de error, se llenarán como null los contenidos que no se puedan descargar
    # El try contiene el algoritmo para descargar la descripción de un proyecto
    try:  
        description = bs.find_all("div", {"class":"rte__content js-full-description responsive-media"})
        # Encontrar todos los párrafos
        description = description[0].find_all("p")
        # Crear array vacío donde se almacenará el contenido descargado
        project_description = []
        # Iteración para agregando en lista cada descripción descargada
        for link in description:
            project_description.append(link.text)
        # Juntar párrafos separados en un solo vector, separándolos con un espacio
        project_description = ' '.join(project_description)
        # Eliminar caracteres especiales
        project_description = project_description.replace(u'\xa0', u'')
        project_description = project_description.replace(u'\n', u' ')
    # Y el except sirve para llenar valores nulos en el array en caso de error
    except (IndexError, ValueError):
        project_description = 'null'
        
    # Eliminar saltos de línea
    return Newlines.sub('\n', project_description)

In [10]:
# Asignar urls de búsqueda (origen)
urls_primera_parte = urls_list[0:9083]
urls_segunda_parte = urls_list[9084:18168]
urls_tercera_parte = urls_list[18169:27251]
#urls

In [11]:
ids_primera_parte = data_final["id"][0:9083]
ids_segunda_parte = data_final["id"][9084:18168]
ids_tercera_parte = data_final["id"][18169:27251]

In [12]:
description_txt = [getPageText(url) for url in urls_primera_parte]

In [13]:
df = {"id":ids_primera_parte, "description":description_txt}

In [17]:
data_final = pd.DataFrame(df)
data_final.head(10)

,id,description
0,1000245024,Purpose Safer Home ensures that electrical plu...
1,1000256230,I once had a Reddit account that was four year...
2,1000261018,Every day you go home to a mail box filled wit...
3,100039820,Funds needed for tooling. Once tooling is done...
4,1000468345,The iRNinja is a wireless 4 button keypad that...
5,1000551777,"Gantish is a web-based platform for parents, w..."
6,1000558429,"Hello, At SellEgg, our goal is to establish co..."
7,1000590709,I want to build a wearable Musical Light Suit ...
8,1000624031,"Hello, my name is Kenneth Steenrod. I have bee..."
9,1000629643,Hello Kickstarter! We are excited to launch an...


In [15]:
export_csv = data_final.to_csv (r'D:\TTT\DB\dataset_descripciones\data_final_parte1.csv', index = None, header=True)

In [11]:
# Hacemos lo mismo con la segunda parte
description_txt = [getPageText(url) for url in urls_segunda_parte]
df = {"id":ids_segunda_parte, "description":description_txt}
data_final = pd.DataFrame(df)
export_csv = data_final.to_csv (r'D:\TTT\DB\dataset_descripciones\data_final_parte2.csv', index = None, header=True)

In [12]:
# Hacemos lo mismo con la tercera parte
description_txt = [getPageText(url) for url in urls_tercera_parte]
df = {"id":ids_tercera_parte, "description":description_txt}
data_final = pd.DataFrame(df)
export_csv = data_final.to_csv (r'D:\TTT\DB\dataset_descripciones\data_final_parte3.csv', index = None, header=True)

In [87]:
# Borramos variables para liberar memoria
del description_txt
del df
del data_final
del export_csv

In [13]:
# Me redirijo a la nueva ruta de los archivos generados
os.chdir('D:/TTT/DB/dataset_descripciones')

In [14]:
#Las siguientes líneas son para combinar todos los archivos .csv dentro de la carpeta
extension = '.csv'
all_filenames = [i for i in glob.glob('*'.format(extension))]

#combine all files in the list
data_description = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
data_description.to_csv( "data_description.csv", index=False, encoding='utf-8-sig')